In [ ]:
from os import environ

from mlrun import import_function, mount_v3io

from stream import Config


fn = import_function("stream.yaml")

# In order to configure the stream processing instance, use the Config object
config = Config(
    project=""
)

fn.set_envs(
    {
        "V3IO_ACCESS_KEY": environ.get("V3IO_ACCESS_KEY"),
        "V3IO_FRAMESD": environ.get("V3IO_FRAMESD"),
        "V3IO_API": environ.get("V3IO_API"),
        **config.as_dict()
    }
)

fn.add_v3io_stream_trigger(
    stream_path=f"projects/{config.project}/model-endpoints/stream",
    name="monitoring_stream_trigger",
)

fn.metadata.project = config.project

fn.spec.no_cache=True

fn.spec.build.commands = [
    "pip install apscheduler",
]

fn.apply(mount_v3io())

fn.deploy()